In [0]:
!pip install --upgrade git+https://github.com/EmGarr/od.git
# Useful for tensorboard
!pip install --upgrade grpcio
!pip install --upgrade tensorflow-gpu

In [0]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Download Pascal VOC

Download and preprocess Pascal VOC to the following format (required by od networks):

```python
dataset = {
        'images' : A tensor of float32 and shape [1, height, widht, 3],
        'images_info': A tensor of float32 and shape [1, 2] ,
        'bbox': A tensor of float32 and shape [1, num_boxes, 4],
        'labels': A tensor of int32 and shape [1, num_boxes],
        'num_boxes': A tensor of int32 and shape [1, 1],
        'weights': A tensor of float32 and shape [1, num_boxes]
    }

```

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
from od.dataset.preprocessing import preprocess, expand_dims_for_single_batch

ds_train, ds_info = tfds.load(name="voc", split="train", shuffle_files=True, with_info=True)
ds_train = ds_train.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = tfds.load(name="voc", split="test", shuffle_files=False)
ds_test = ds_test.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
ds_info

# Load and train the network


In [0]:
from od.model.faster_rcnn import build_fpn_resnet50_faster_rcnn
from od.core.standard_fields import BoxField
from od.core.learning_rate_schedule import WarmupLearningRateScheduler
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.mixed_precision import experimental as mixed_precision

# Number of classes of Pascal Voc
num_classes = 20
batch_size = 1

model_faster_rcnn = build_fpn_resnet50_faster_rcnn(num_classes, batch_size)
base_lr = 0.02
optimizer = tf.keras.optimizers.SGD(learning_rate=base_lr)
model_faster_rcnn.compile(optimizer=optimizer, loss=None)
callbacks = [WarmupLearningRateScheduler(base_lr, 1, epochs=[8, 10], init_lr=0.0001), TensorBoard(), ModelCheckpoint('.checkpoints/')]

model_faster_rcnn.fit(ds_train, validation_data=ds_test, epochs=11, callbacks=callbacks)

# Tensorboard

In [0]:
# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir logs